## TODO
- titleのembedding
- tokenizer, pretrained-embeddingの利用
- modelの中での読み込み

In [54]:
import pandas as pd
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import os, sys, gc, random, multiprocessing, glob, time

DATA_DIR = '../input/google-quest-challenge'
# DATA_DIR = 'D:/project/ICF_AutoCapsule_disabled/kaggle/google-quest-challenge'
# BERT_DIR = 'D:/project/ICF_AutoCapsule_disabled/BERT'

In [55]:
# import nltk
# nltk.download('averaged_perceptron_tagger')

In [56]:
!pip install ../input/sacremoses/sacremoses-master/
!pip install ../input/transformers/transformers-master/

# !pip install ../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec
# !pip install ../input/glove840b300dtxt/glove.840B.300d.txt

CRAWL_EMBEDDING_PATH = '../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec'
GLOVE_EMBEDDING_PATH = '../input/glove840b300dtxt/glove.840B.300d.txt'
# CRAWL_EMBEDDING_PATH = 'D:/project/ICF_AutoCapsule_disabled/W2V/crawl-300d-2M.vec'
# GLOVE_EMBEDDING_PATH = 'D:/project/ICF_AutoCapsule_disabled/W2V/glove.840B.300d.txt'

In [57]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf

from nlpaug.util import Action

In [58]:
import keras

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset

#from ml_stratifiers import MultilabelStratifiedShuffleSplit, MultilabelStratifiedKFold
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder

from scipy.stats import spearmanr

import transformers
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,
)

from tqdm import tqdm
print(transformers.__version__)

2.3.0


In [59]:
## Make results reproducible .Else noone will believe you .
import random

def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [60]:
class PipeLineConfig:
    def __init__(self, lr, warmup, epochs, patience, batch_size, seed, name, question_weight,answer_weight,fold,train):
        self.lr = lr
        self.warmup = warmup
        self.epochs = epochs
        self.patience = patience
        self.batch_size = batch_size
        self.seed = seed
        self.name = name
        self.question_weight = question_weight
        self.answer_weight =answer_weight
        self.fold = fold
        self.train = train

In [61]:
config = PipeLineConfig(lr=1e-5, \
                        warmup=0.01, \
                        epochs=4, \
                        patience=3, \
                        batch_size=8, \
                        seed=42, \
                        name='remodel_title0115', \
                        question_weight=0.5, \
                        answer_weight=0.5, \
                        fold=5, \
                        train=True
                       )

In [62]:
seed_everything(config.seed)

In [63]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
print(device)

cuda


In [64]:
sub = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')
sub.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,...,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308
1,46,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,...,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448
2,70,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,...,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673
3,132,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,...,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401
4,200,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,...,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074


In [65]:
target_columns = sub.columns.values[1:].tolist()
target_columns

['question_asker_intent_understanding',
 'question_body_critical',
 'question_conversational',
 'question_expect_short_answer',
 'question_fact_seeking',
 'question_has_commonly_accepted_answer',
 'question_interestingness_others',
 'question_interestingness_self',
 'question_multi_intent',
 'question_not_really_a_question',
 'question_opinion_seeking',
 'question_type_choice',
 'question_type_compare',
 'question_type_consequence',
 'question_type_definition',
 'question_type_entity',
 'question_type_instructions',
 'question_type_procedure',
 'question_type_reason_explanation',
 'question_type_spelling',
 'question_well_written',
 'answer_helpful',
 'answer_level_of_information',
 'answer_plausible',
 'answer_relevance',
 'answer_satisfaction',
 'answer_type_instructions',
 'answer_type_procedure',
 'answer_type_reason_explanation',
 'answer_well_written']

In [66]:
train = pd.read_csv(f'{DATA_DIR}/train.csv')
train.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,...,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,...,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [67]:
test = pd.read_csv(f'{DATA_DIR}/test.csv')
test.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [68]:
test.question_title.apply(lambda x: len(x.split())).quantile(0.99)

17.0

In [69]:
train.question_body.apply(lambda x: len(x.split())).quantile(0.9)

246.0

## Preprocessing

In [70]:
import re
from flashtext import KeywordProcessor

In [71]:
PUNCTS = {
            '》', '〞', '¢', '‹', '╦', '║', '♪', 'Ø', '╩', '\\', '★', '＋', 'ï', '<', '?', '％', '+', '„', 'α', '*', '〰', '｟', '¹', '●', '〗', ']', '▾', '■', '〙', '↓', '´', '【', 'ᴵ',
            '"', '）', '｀', '│', '¤', '²', '‡', '¿', '–', '」', '╔', '〾', '%', '¾', '←', '〔', '＿', '’', '-', ':', '‧', '｛', 'β', '（', '─', 'à', 'â', '､', '•', '；', '☆', '／', 'π',
            'é', '╗', '＾', '▪', ',', '►', '/', '〚', '¶', '♦', '™', '}', '″', '＂', '『', '▬', '±', '«', '“', '÷', '×', '^', '!', '╣', '▲', '・', '░', '′', '〝', '‛', '√', ';', '】', '▼',
            '.', '~', '`', '。', 'ə', '］', '，', '{', '～', '！', '†', '‘', '﹏', '═', '｣', '〕', '〜', '＼', '▒', '＄', '♥', '〛', '≤', '∞', '_', '[', '＆', '→', '»', '－', '＝', '§', '⋅', 
            '▓', '&', 'Â', '＞', '〃', '|', '¦', '—', '╚', '〖', '―', '¸', '³', '®', '｠', '¨', '‟', '＊', '£', '#', 'Ã', "'", '▀', '·', '？', '、', '█', '”', '＃', '⊕', '=', '〟', '½', '』',
            '［', '$', ')', 'θ', '@', '›', '＠', '｝', '¬', '…', '¼', '：', '¥', '❤', '€', '−', '＜', '(', '〘', '▄', '＇', '>', '₤', '₹', '∅', 'è', '〿', '「', '©', '｢', '∙', '°', '｜', '¡', 
            '↑', 'º', '¯', '♫', '#'
          }


mispell_dict = {"aren't" : "are not", "can't" : "cannot", "couldn't" : "could not",
"couldnt" : "could not", "didn't" : "did not", "doesn't" : "does not",
"doesnt" : "does not", "don't" : "do not", "hadn't" : "had not", "hasn't" : "has not",
"haven't" : "have not", "havent" : "have not", "he'd" : "he would", "he'll" : "he will", "he's" : "he is", "i'd" : "I would",
"i'd" : "I had", "i'll" : "I will", "i'm" : "I am", "isn't" : "is not", "it's" : "it is",
"it'll":"it will", "i've" : "I have", "let's" : "let us", "mightn't" : "might not", "mustn't" : "must not", 
"shan't" : "shall not", "she'd" : "she would", "she'll" : "she will", "she's" : "she is", "shouldn't" : "should not", "shouldnt" : "should not",
"that's" : "that is", "thats" : "that is", "there's" : "there is", "theres" : "there is", "they'd" : "they would", "they'll" : "they will",
"they're" : "they are", "theyre":  "they are", "they've" : "they have", "we'd" : "we would", "we're" : "we are", "weren't" : "were not",
"we've" : "we have", "what'll" : "what will", "what're" : "what are", "what's" : "what is", "what've" : "what have", "where's" : "where is",
"who'd" : "who would", "who'll" : "who will", "who're" : "who are", "who's" : "who is", "who've" : "who have", "won't" : "will not", "wouldn't" : "would not", "you'd" : "you would",
"you'll" : "you will", "you're" : "you are", "you've" : "you have", "'re": " are", "wasn't": "was not", "we'll":" will", "didn't": "did not", "tryin'":"trying"}


kp = KeywordProcessor(case_sensitive=True)
for k, v in mispell_dict.items():
    kp.add_keyword(k, v)

def clean_punct(text):
    text = str(text)
    for punct in PUNCTS:
        text = text.replace(punct, ' {} '.format(punct))
    return text


def preprocessing(text):
    text = text.lower()
    text = re.sub(r'(\&lt)|(\&gt)', ' ', text)
    
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' url ', text)
    text = kp.replace_keywords(text)
    text = clean_punct(text)
    text = re.sub(r'\n\r', ' ', text)
    text = re.sub(r'\s{2,}', ' ', text)
    
    return text

In [72]:
# train['question_title'] = train['question_title'].apply(lambda x : preprocessing(x))
# train['question_body'] = train['question_body'].apply(lambda x : preprocessing(x))
# train['answer'] = train['answer'].apply(lambda x : preprocessing(x))

# test['question_title'] = test['question_title'].apply(lambda x : preprocessing(x))
# test['question_body'] = test['question_body'].apply(lambda x : preprocessing(x))
# test['answer'] = test['answer'].apply(lambda x : preprocessing(x))

In [73]:
# # category, host変数を、数値化(LabelEncode)
# le_category = LabelEncoder()
# le_category.fit(np.concatenate([train.category.values, test.category.values], axis=0))
# train['category'] = le_category.transform(train.category)
# test['category'] = le_category.transform(test.category)

# le_host = LabelEncoder()
# le_host.fit(np.concatenate([train.host.values, test.host.values], axis=0))
# train['host'] = le_host.transform(train.host)
# test['host'] = le_host.transform(test.host)

In [74]:
category_list = ['CULTURE', 'LIFE_ARTS', 'SCIENCE', 'STACKOVERFLOW', 'TECHNOLOGY']

host_list = ['academia.stackexchange.com', 'android.stackexchange.com',
       'anime.stackexchange.com', 'apple.stackexchange.com',
       'askubuntu.com', 'bicycles.stackexchange.com',
       'biology.stackexchange.com', 'blender.stackexchange.com',
       'boardgames.stackexchange.com', 'chemistry.stackexchange.com',
       'christianity.stackexchange.com', 'codereview.stackexchange.com',
       'cooking.stackexchange.com', 'crypto.stackexchange.com',
       'cs.stackexchange.com', 'dba.stackexchange.com',
       'diy.stackexchange.com', 'drupal.stackexchange.com',
       'dsp.stackexchange.com', 'electronics.stackexchange.com',
       'ell.stackexchange.com', 'english.stackexchange.com',
       'expressionengine.stackexchange.com', 'gamedev.stackexchange.com',
       'gaming.stackexchange.com', 'gis.stackexchange.com',
       'graphicdesign.stackexchange.com', 'judaism.stackexchange.com',
       'magento.stackexchange.com', 'math.stackexchange.com',
       'mathematica.stackexchange.com', 'mathoverflow.net',
       'mechanics.stackexchange.com', 'meta.askubuntu.com',
       'meta.christianity.stackexchange.com',
       'meta.codereview.stackexchange.com', 'meta.math.stackexchange.com',
       'meta.stackexchange.com', 'meta.superuser.com',
       'money.stackexchange.com', 'movies.stackexchange.com',
       'music.stackexchange.com', 'photo.stackexchange.com',
       'physics.stackexchange.com', 'programmers.stackexchange.com',
       'raspberrypi.stackexchange.com', 'robotics.stackexchange.com',
       'rpg.stackexchange.com', 'salesforce.stackexchange.com',
       'scifi.stackexchange.com', 'security.stackexchange.com',
       'serverfault.com', 'sharepoint.stackexchange.com',
       'softwarerecs.stackexchange.com', 'stackoverflow.com',
       'stats.stackexchange.com', 'superuser.com',
       'tex.stackexchange.com', 'travel.stackexchange.com',
       'unix.stackexchange.com', 'ux.stackexchange.com',
       'webapps.stackexchange.com', 'webmasters.stackexchange.com',
       'wordpress.stackexchange.com']

## DataAugumentation
- どの程度の割合で、行うか？
- 基本的に、ラベルは変えたくないので、意味合いを変えないようにAugmetationしたい
- 全てを水増しすると、データが単純に倍になってしまうので、ある程度ランダムにいくつかだけ増やすもしくは変える、変えない

In [75]:
#BERT Augmentator
aug_bert = naw.ContextualWordEmbsAug(
    #model_path=BERT_DIR+'/bert-base-uncased',
    model_path='bert-base-uncased',
    device='cuda',
    action='insert', #"substitute"
    aug_p=0.3, # 含まれているwordsの中の何割を変換するか
    temperature=0.3 , # 変換を施すかどうかの確率
    top_k=10)

# text = 'The quick brown fox jumps over the lazy dog .'
# print("original: \n", text)
# for _ in range(10):
#     augmented_text = aug_bert.augment(text)
#     print(augmented_text)

In [76]:
# # 上位語や下位語で、文字が置換
# aug_syn = naw.SynonymAug(
#     aug_src='wordnet', 
#     aug_p=0.3, 
# )

## Word Embedding
- BERTのtokenizerで、単語は番号付け
- 学習済み単語のembeddingと、Tokenizerの番号の対応づけを行う

In [77]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')

def load_embeddings(path):
    with open(path, encoding='utf-8') as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in tqdm(f))

def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words

In [78]:
#Tokenizer = BertTokenizer.from_pretrained(BERT_DIR+'/bert-base-uncased')
#Tokenizer = BertTokenizer.from_pretrained('../input/bert-base-uncased/')
#Tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokneizerの学習
Tokenizer = keras.preprocessing.text.Tokenizer(filters='', lower=False)
Tokenizer.fit_on_texts(list(train.question_title) + list(train.question_body) + list(train.answer) \
                        + list(test.question_title) + list(test.question_body) + list(test.answer))

import pickle
# # saving
# with open('../input/tokenizer/tokenizer.pickle', 'wb') as handle:
#     pickle.dump(Tokenizer, handle)

# loading
with open('../input/tokenizer/tokenizer.pickle', 'rb') as handle:
    Tokenizer = pickle.load(handle)

In [21]:
Tokenizer.texts_to_sequences([train.question_title[0]])[0]

[115, 63, 7, 3027, 45, 51, 1425, 11683, 281, 4, 3, 3206, 7229]

In [51]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
ids = tokenizer.tokenize(train.question_title[0])
ids

['what',
 'am',
 'i',
 'losing',
 'when',
 'using',
 'extension',
 'tubes',
 'instead',
 'of',
 'a',
 'macro',
 'lens',
 '?']

In [52]:
tokenizer.convert_tokens_to_ids(ids)

[2054,
 2572,
 1045,
 3974,
 2043,
 2478,
 5331,
 10868,
 2612,
 1997,
 1037,
 26632,
 10014,
 1029]

In [ ]:
crawl_matrix, unknown_words_crawl = build_matrix(Tokenizer.word_index, CRAWL_EMBEDDING_PATH)
print('n unknown words (crawl): ', len(unknown_words_crawl))

In [ ]:
glove_matrix, unknown_words_glove = build_matrix(Tokenizer.word_index, GLOVE_EMBEDDING_PATH)
print('n unknown words (glove): ', len(unknown_words_glove))

In [ ]:
embedding_matrix = crawl_matrix + glove_matrix  
embedding_matrix.shape

del crawl_matrix
del glove_matrix
gc.collect()

## Dataset

In [155]:
MAX_LEN = 512

MAX_T_LEN = 17
MAX_Q_LEN = 250
MAX_A_LEN = 259
SEP_TOKEN_ID = 102 # bert-base-uncasedにおけるvocabの'[SEP']が、102番目という意味

class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, df, train_mode=True, labeled=True):
        self.df = df
        self.train_mode = train_mode
        self.labeled = labeled
        #self.tokenizer = BertTokenizer.from_pretrained(BERT_DIR+'/bert-base-uncased')
        self.tokenizer = BertTokenizer.from_pretrained('../input/bertbaseuncased/')
        #self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __getitem__(self, index):
        """
        token_id列
        segment_id列
        label列
        """
        row = self.df.iloc[index]
        token_ids, seg_ids, t_token_ids = self.get_token_ids(row)
     
        if self.labeled:
            labels = self.get_label(row)
            return token_ids, seg_ids, t_token_ids, torch.tensor(row.category), torch.tensor(row.host), labels
        else:
            return token_ids, seg_ids, t_token_ids, torch.tensor(row.category), torch.tensor(row.host)

    def __len__(self):
        return len(self.df)


#     def select_tokens(self, tokens, max_num):
#         if len(tokens) <= max_num:
#             return tokens
#         if self.train_mode:
#             num_remove = len(tokens) - max_num
#             remove_start = random.randint(0, len(tokens)-num_remove-1)
#             return tokens[:remove_start] + tokens[remove_start + num_remove:]
#         else:
#             return tokens[:max_num//2] + tokens[-(max_num - max_num//2):]

    def trim_input(self, title, question, answer, max_sequence_length=MAX_LEN, 
                t_max_len=MAX_T_LEN, q_max_len=MAX_Q_LEN, a_max_len=MAX_A_LEN):
        """
        title. question, answerそれぞれのセンテンスを、tokenizeする
        lengthが長い場合の処理
        """
        t = Tokenizer.texts_to_sequences([title])#[0]
        
        q = self.tokenizer.tokenize(question)
        a = self.tokenizer.tokenize(answer)

        t_len = len(t)
        q_len = len(q)
        a_len = len(a)
        
        if t_len > t_max_len:
            t_new_len = t_max_len
        else:
            t_new_len = t_len
        
        if (q_len+a_len+3) > max_sequence_length:
            if q_len < q_max_len:
                q_new_len = q_len
                a_new_len = a_len - ((q_len+a_len+3) - max_sequence_length)
            elif a_len < a_max_len:
                q_new_len = q_len - ((q_len+a_len+3) - max_sequence_length)
                a_new_len = a_len
            else:
                q_new_len = q_max_len
                a_new_len = a_max_len
                

            if a_new_len+q_new_len+3 != max_sequence_length:
                raise ValueError("New sequence length should be %d, but is %d" 
                                 % (max_sequence_length, (a_new_len+q_new_len+3)))

            t = t[:t_new_len]
            q = q[:q_new_len]
            a = a[:a_new_len]

        return t, q, a
        
    def get_token_ids(self, row):
#         question_title = aug_bert.augment(row.question_title)
#         question_body = aug_bert.augment(row.question_body)
#         answer = aug_bert.augment(row.answer)
        
        question_title = row.question_title
        question_body = row.question_body
        answer = row.answer
    
        t_tokens, q_tokens, a_tokens = self.trim_input(question_title, question_body, answer)
        
        # BERTの入力タイプに変換([CLS]と[SEP]をつないで、１つのsetentenceに)
        tokens = ['[CLS]'] + q_tokens + ['[SEP]'] + a_tokens + ['[SEP]']
        token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        if len(token_ids) < MAX_LEN:
            """0で後ろからpadding"""
            token_ids += [0] * (MAX_LEN - len(token_ids))
        ids = torch.tensor(token_ids)
        seg_ids = self.get_seg_ids(ids)  # segment_embを区別するindex
        
        if len(t_tokens) < MAX_T_LEN:
            t_tokens = keras.preprocessing.sequence.pad_sequences(t_tokens, maxlen=MAX_T_LEN, padding='post')
        t_tokens = torch.tensor(t_tokens, dtype=torch.int64).squeeze()
        
        return ids, seg_ids, t_tokens
    
    def get_seg_ids(self, ids):
        """
        いくつめの文かを区別するsegment_idを、各文字に振る
        """
        seg_ids = torch.zeros_like(ids) # [max_len]のtorch_tensor
        seg_idx = 0
        first_sep = True
        for i, e in enumerate(ids):
            seg_ids[i] = seg_idx
            if e == SEP_TOKEN_ID: # [SEP]の場合
                seg_idx = 1
#         pad_idx = torch.nonzero(ids == 0)  # bert-base_uncasedのvocabで、[PAD]は0番目であるので、PADの部分のindexだけ抽出
#         seg_ids[pad_idx] = 0

        return seg_ids

    def get_label(self, row):
        #print(row[target_columns].values)
        return torch.tensor(row[target_columns].values.astype(np.float32))

    def collate_fn(self, batch):
        """
        labelデータを持つモードと、ない完全な推論モードでは、batchのshapeが異なるので(labelが2番目の要素にあるなし)
        """        
        token_ids = torch.stack([x[0] for x in batch])
        seg_ids = torch.stack([x[1] for x in batch])
        t_token_ids = torch.stack([x[2] for x in batch])
        
        category = torch.stack([x[-3] for x in batch])
        host = torch.stack([x[-2] for x in batch])
    
        if self.labeled:
            labels = torch.stack([x[-1] for x in batch])
            return token_ids, seg_ids, t_token_ids, category, host, labels
        else:
            return token_ids, seg_ids, t_token_ids, category, host

In [45]:
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0):
    df = pd.read_csv(f'{DATA_DIR}/train.csv')
        
    # cleaning
    df['question_title'] = df['question_title'].apply(lambda x : preprocessing(x))
    df['question_body'] = df['question_body'].apply(lambda x : preprocessing(x))
    df['answer'] = df['answer'].apply(lambda x : preprocessing(x))
        
    # label encode
    le_category = LabelEncoder()
    le_category.fit(category_list)
    for c in set(df.category):
        if c not in category_list:
            df.category = df.category.replace(c, np.nan)
            df.category = df.category.fillna(train.category.mode()[0])
    df.category = le_category.transform(df.category)
    
    le_host = LabelEncoder()
    le_host.fit(host_list)
    for c in set(df.host):
        if c not in host_list:
            df.host = df.host.replace(c, np.nan)
            df.host = df.host.fillna(train.host.mode()[0])
    df.host = le_host.transform(df.host)
    
    
    df = shuffle(df, random_state=1234)
    gkf = GroupKFold(n_splits=5).split(X=df.question_body, groups=df.question_body)
    for fold, (train_idx, valid_idx) in enumerate(gkf):
        if fold == ifold:
            df_train = df.iloc[train_idx]
            df_val = df.iloc[valid_idx]
            break

    print('train', df_train.shape)
    print('val', df_val.shape)

    ds_train = QuestDataset(df_train, train_mode=True)
    train_loader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, shuffle=True, num_workers=0, collate_fn=ds_train.collate_fn, drop_last=True)
    train_loader.num = len(df_train)

    ds_val = QuestDataset(df_val, train_mode=False)
    val_loader = torch.utils.data.DataLoader(ds_val, batch_size=val_batch_size, shuffle=False, num_workers=0, collate_fn=ds_val.collate_fn, drop_last=False)
    val_loader.num = len(df_val)
    val_loader.df = df_val

    return train_loader, val_loader, df_val.shape[0], valid_idx


def get_test_loader(batch_size=4):
    df = pd.read_csv(f'{DATA_DIR}/test.csv')
    
    # cleaning
    df['question_title'] = df['question_title'].apply(lambda x : preprocessing(x))
    df['question_body'] = df['question_body'].apply(lambda x : preprocessing(x))
    df['answer'] = df['answer'].apply(lambda x : preprocessing(x))
        
    # label encode
    le_category = LabelEncoder()
    le_category.fit(category_list)
    for c in set(df.category):
        if c not in category_list:
            df.category = df.category.replace(c, np.nan)
            df.category = df.category.fillna(train.category.mode()[0])
    df.category = le_category.transform(df.category)
    
    le_host = LabelEncoder()
    le_host.fit(host_list)
    for c in set(df.host):
        if c not in host_list:
            df.host = df.host.replace(c, np.nan)
            df.host = df.host.fillna(train.host.mode()[0])
    df.host = le_host.transform(df.host)
    
    
    ds_test = QuestDataset(df, train_mode=False, labeled=False)
    loader = torch.utils.data.DataLoader(ds_test, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=ds_test.collate_fn, drop_last=False)
    loader.num = len(df)
    
    return loader


# def test_train_loader():
#     loader, _ = get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=1)
#     for ids, seg_ids, labels in loader:
#         print(ids)
#         print(seg_ids.numpy())
#         print(labels)
#         break
# def test_test_loader():
#     loader = get_test_loader(4)
#     for ids, seg_ids in loader:
#         print(ids)
#         print(seg_ids)
#         break

In [ ]:
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x

In [ ]:
class QuestModel(nn.Module):
    def __init__(self, lstm_hidden_unit=32, num_layer=1, n_classes=30):
        super(QuestModel, self).__init__()
        self.model_name = 'QuestModel'
#         self.bert_model1 = BertModel.from_pretrained(BERT_DIR+'/bert-base-uncased/')
#         self.bert_model2 = BertModel.from_pretrained(BERT_DIR+'/bert-base-uncased/')
        
        self.bert_model = BertModel.from_pretrained('../input/bertbaseuncased/')
        
        #self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        #self.bert_model2 = BertModel.from_pretrained('bert-base-uncased')
        
        # word embedding
        self.num_words = embedding_matrix.shape[0]
        self.embedding_size = embedding_matrix.shape[1]
        self.embedding = nn.Embedding(self.num_words, self.embedding_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.required_grad = False
        
        #self.embedding_dropout = SpatialDropout(0.4)
        
        #self.lstm = nn.LSTM(self.embedding_size, lstm_hidden_unit, num_layers=num_layer, bidirectional=True, batch_first=True)        
        #self.emb_title = nn.Embedding(MAX_TITLE_LEN, 10)
        
        self.emb_category = nn.Embedding(len(category_list), 3)
        self.emb_host = nn.Embedding(len(host_list), 10)
        
        self.fc_emb = nn.Linear(300+3+10, 100)
        self.fc = nn.Linear(768+100, n_classes)

    def forward(self, ids, seg_ids, t_token_ids, category, host):
        
        attention_mask = (ids > 0)  # ids==0([PAD])部分だけFalseとなるので、そこだけattention_weightを0に
        layers, pool_out = self.bert_model(input_ids=ids, token_type_ids=seg_ids, attention_mask=attention_mask)
        
        out = F.avg_pool1d(layers.transpose(1,2), kernel_size=layers.size()[1]).squeeze()
        
#         emb_title = self.emb_title(ids_title)
#         emb_title = F.max_pool1d(emb_title.transpose(1,2), kernel_size=emb_title.size()[1]).squeeze()
        
    
        out_t = self.embedding(t_token_ids)
        out_t = F.dropout(out_t, p=0.4, training=self.training)
        #out_t, _ = self.lstm(out_t)
        out_t = F.avg_pool1d(out_t.transpose(1,2), kernel_size=out_t.size()[1]).squeeze()
        #print(out_t.size())
        
        emb_category = self.emb_category(category)
        emb_host = self.emb_host(host)
        
        out_t = torch.cat([out_t, emb_category], dim=-1)
        out_t = torch.cat([out_t, emb_host], dim=-1)
        out_t = F.dropout(out_t, p=0.2, training=self.training)
        out_t = F.relu(self.fc_emb(out_t))
        
        out = torch.cat([out, out_t], dim=-1)
        
        out = F.dropout(out, p=0.2, training=self.training)
        logit = self.fc(out)
        return logit # 単に30種類の出力値を算出

In [ ]:
def train_model(train_loader, optimizer, criterion, scheduler):
    model.train()
    avg_loss = 0.    
    for idx, batch in enumerate(tqdm(train_loader)):
        ids_train, seg_ids_train, t_ids_train, category_train, host_train, label_ids_train = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device), batch[4].to(device), batch[5].to(device)
        
        #print(host_train)
        
        logits = model(ids_train, seg_ids_train, t_ids_train, category_train, host_train)
        #logits = torch.sigmoid(model(ids_train, seg_ids_train))
        
        loss = config.question_weight*criterion(logits[:,0:21], label_ids_train[:,0:21]) + config.answer_weight*criterion(logits[:,21:30], label_ids_train[:,21:30])
        
        
#         loss1 = criterion(logits[:,0:9], labels[:,0:9])
#         loss2 = criterion(logits[:,9:10], labels[:,9:10])
#         loss3 = criterion(logits[:,10:21], labels[:,10:21])
#         loss4 = criterion(logits[:,21:26], labels[:,21:26])
#         loss5 = criterion(logits[:,26:30], labels[:,26:30])
#         loss = config.question_weight*loss1+config.answer_weight*loss2+config.question_weight*loss3+config.answer_weight*loss4+config.question_weight*loss5
        
        
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        avg_loss += loss.item() / len(train_loader)
        del ids_train, seg_ids_train, label_ids_train

    torch.cuda.empty_cache()
    gc.collect()
    return avg_loss

def val_model(val_loader, val_length, batch_size=8):
    model.eval() # eval mode  
    avg_val_loss = 0.
    
    valid_preds = np.zeros((val_length, len(target_columns)))
    original = np.zeros((val_length, len(target_columns)))
    
    with torch.no_grad():
        for idx, batch in enumerate(tqdm(val_loader)):
            ids, seg_ids, t_ids_val, category_val, host_val, labels = batch[0].to(device), batch[1].to(device), batch[2].to(device), batch[3].to(device), batch[4].to(device), batch[5].to(device)
            
            logits = model(ids, seg_ids, t_ids_val, category_val, host_val)
            
            avg_val_loss += criterion(logits, labels).item() / len(val_loader)
            valid_preds[idx*batch_size : (idx+1)*batch_size] = torch.sigmoid(logits).detach().cpu().squeeze().numpy()
            original[idx*batch_size : (idx+1)*batch_size]    = labels.detach().cpu().squeeze().numpy()
        
        score = 0
        preds = torch.tensor(valid_preds).numpy()
        #preds = torch.sigmoid(torch.tensor(valid_preds)).numpy()
        
        rho_val = np.mean([spearmanr(original[:, i], preds[:,i]).correlation for i in range(preds.shape[1])])
        print('\r val_spearman-rho: %s' % (str(round(rho_val, 5))), end = 100*' '+'\n')
        
        for i in range(len(target_columns)):
            print(i, spearmanr(original[:,i], preds[:,i]))
            score += np.nan_to_num(spearmanr(original[:, i], preds[:, i]).correlation, nan=0)
    
    return avg_val_loss, score/len(target_columns)

In [ ]:
def calc_spearman(pred, target):
    score = 0
    for i in range(len(target)):
        #print(i, spearmanr(target[:,i], preds[:,i]))
        score += np.nan_to_num(spearmanr(target[:, i], pred[:, i]).correlation, nan=0.5)
    return score/len(target)

In [ ]:
ACCUM_STEPS = 1

In [ ]:
def create_model(model_file):
    model = QuestModel(n_classes=30).to(device)
    model.load_state_dict(torch.load(model_file))
    model = model
    #model = DataParallel(model)
    return model

def create_models():
    models = []
    for fold in range(config.fold):
        model = create_model('../input/models6/models_best_param_score_{}_{}.pt'.format(config.name,(fold+1)))
        model.eval()
        models.append(model)
    return models

In [ ]:
def predict(models, test_loader):
    all_scores = []
    with torch.no_grad():
        for ids, seg_ids, category, host in tqdm(test_loader, total=test_loader.num // test_loader.batch_size):
            ids, seg_ids, category, host = ids.cuda(), seg_ids.cuda(), category.cuda(), host.cuda()
            scores = []
            for model in models:
                outputs = torch.sigmoid(model(ids, seg_ids, category, host)).cpu()
                scores.append(outputs)
            all_scores.append(torch.mean(torch.stack(scores), 0))

    all_scores = torch.cat(all_scores, 0).numpy()
    
    return all_scores

In [ ]:
test_loader = get_test_loader(batch_size=32)
models = create_models()

preds = predict(models, test_loader)

In [ ]:
sub[target_columns] = preds
sub.to_csv('submission.csv', index=False)
sub.head()